## RunnableWithMessageHistory

https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [1]:
from operator import itemgetter
# from langchain_openai.chat_models import ChatOpenAI
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: list[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]


history = get_by_session_id("1")
history.add_message(AIMessage(content="hello"))
print(store)  # noqa: T201

{'1': InMemoryHistory(messages=[AIMessage(content='hello', additional_kwargs={}, response_metadata={})])}


In [19]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_chroma import Chroma

ollama_llm = OllamaLLM(
    base_url='http://dandelion-ollama-1:11434', 
    model="llama3.1:8b",
    temperature=0.0,
    num_predict=51200
)

In [3]:
# from langchain.chains import ConversationChain # deplicate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# memory 管理器
memory = ConversationBufferMemory(return_messages=True)
# conversation = ConversationChain(
#     llm=ollama_llm,
#     memory=ConversationBufferMemory()
# )

# 定義 prompt 模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

# 建立 chain
chain = prompt | ollama_llm
chain_with_history = RunnableWithMessageHistory(
    chain,
    # Uses the get_by_session_id function defined in the example
    # above.
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

/tmp/ipykernel_13392/3966929422.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 載入 PDF 文件
loader = PyPDFLoader('./data/PDF_file.pdf')
docs = loader.load_and_split()

chunk_size = 256
chunk_overlap = 128

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
documents = text_splitter.split_documents(docs)


In [5]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# 使用 ollama 的 embedding 模型
embeddings_model = OllamaEmbeddings(
    base_url='http://dandelion-ollama-1:11434', 
    model="bge-m3:567m",
)

# 建立 Chroma Vector Store
vector_db = Chroma.from_documents(
    documents,
    embedding=embeddings_model,
    persist_directory="./story-db"
)

In [7]:
query = "你看過經典故事小王子嘛?"

retrieved_docs = vector_db.similarity_search(query, k=5)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [9]:
retrieved_docs

[Document(id='bb8c854f-fd00-4ad0-a2be-07ec344f54b4', metadata={'source': './data/PDF_file.pdf', 'author': 'user', 'creator': 'PrimoPDF http://www.primopdf.com', 'page': 53, 'title': '(Microsoft Word - LE PETIT PRINCE\\244\\244\\244\\345\\252\\251.doc)', 'page_label': '54', 'producer': 'AFPL Ghostscript 8.13', 'creationdate': 'D:20090501171152', 'total_pages': 54, 'moddate': 'D:20090501171152'}, page_content='圖，可以制作一份 WWW 版的《小王子》主頁，那就更好了。 \n \n    《小王子》 是他作品中比較獨特的一篇，也是最著名的一篇。是一本“為大 \n人們寫的童話故事”。我很喜歡它，從小學看到大學。確實，它給大人們看的， \n理解它必須是“大人”才行。可是，我多么希望我還是一個小王子一樣的孩子… \n \n                                       Loking 錄入于 3.15.1997 \n \n◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎\n◎'),
 Document(id='7b548b58-4ab3-490a-bc83-2aac22b7c54b', metadata={'producer': 'AFPL Ghostscript 8.13', 'creator': 'PrimoPDF http://www.primopdf.com', 'page': 53, 'title': '(Microsoft Word - LE PETIT PRINCE\\244\\244\\244\\345\\252\\251.doc)', 'moddate': 'D:20090501171152', 'author': 'user', 'total_pages': 54, 'page_label': '54',

In [8]:

question = "根據以下內容回答問題：\n" + retrieved_context+ "\n\n問題："+ query

print(chain_with_history.invoke(  # noqa: T201
    {
        "ability": "story",
        "question": question
    }, config={"configurable": {"session_id": "foo"}}
    
))

我看過《小王子》！這是個非常著名的小說，作者是安托瓦內·德·聖-埃克絮佩里。他在書中描述了一個孤獨的小王子，他的父親是一個王國的國王，小王子被送到月球上去照顧一隻太陽花。小王子的故事充滿著哲理和寓意，讓人們思考生命的意義、愛情和友誼等問題。

我很喜歡這本書，因為它不僅是一個童話故事，也是對大人們的啟蒙。它教會我們要珍惜生命、關心他人和追求自己的夢想。小王子的故事也讓我感受到孤獨和寂寞的痛苦，但同時也給了我希望和鼓勵。

你看過《小王子》嗎？


In [10]:
query = "啟蒙了你什麼?"

retrieved_docs = vector_db.similarity_search(query, k=5)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

question = "根據以下內容回答問題：\n" + retrieved_context+ "\n\n問題："+ query

print(chain_with_history.invoke(  # noqa: T201
    {
        "ability": "story",
        "question": question
    }, config={"configurable": {"session_id": "foo"}}
    
))

小王子和這個故事都啟蒙了我對生命、愛情和友誼的看法。它們教會我要珍惜生命、關心他人和追求自己的夢想。


In [16]:
query = "啟蒙了你什麼?"

retrieved_docs = vector_db.similarity_search(query, k=5)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

question = "根據以下內容回答問題：\n" + retrieved_context+ "\n\n問題："+ query

print(chain_with_history.invoke(  # noqa: T201
    {
        "ability": "story",
        "question": question
    }, config={"configurable": {"session_id": "foo"}}
    
))

這個故事啟蒙了我對人性的看法。它讓我了解到，大多數大人們都缺乏真實的理解力和創造性思維能力。雖然他們可能有頭腦清晰，但卻無法真正地理解和欣賞藝術作品或哲理思想。這使我更加珍惜自己的獨特性和創造力，同時也讓我更願意與那些有同樣想法的人相遇和交流。


In [17]:
query = "你覺得小王子是個怎樣的人?"

retrieved_docs = vector_db.similarity_search(query, k=5)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

question = "根據以下內容回答問題：\n" + retrieved_context+ "\n\n問題："+ query

print(chain_with_history.invoke(  # noqa: T201
    {
        "ability": "story",
        "question": question
    }, config={"configurable": {"session_id": "foo"}}
    
))

小王子是一個獨特、敏感和浪漫的年輕人。他對世界的看法和感受與一般人不同，他能夠看到星球的美麗和價值，並且願意為之付出努力。然而，他也有一些缺點，例如缺乏勇氣承認自己的情感和關係。


In [20]:
query = "依照這部小王子中提供的內容與你的創造力，續寫第二部小王子。"

retrieved_docs = vector_db.similarity_search(query, k=5)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

question = "根據以下內容回答問題：\n" + retrieved_context+ "\n\n問題："+ query

print(chain_with_history.invoke(  # noqa: T201
    {
        "ability": "story",
        "question": question
    }, config={"configurable": {"session_id": "foo"}}
    
))

根據小王子的故事，我們可以繼續他的冒險旅程。

在小王子和我一起坐下後，他說：“你知道，我一直想去探索更大的世界。”我問他：“為什麼？”他回答：“因為我的星球太小了，我想看看外面的世界有多大多美麗。”

於是，我們決定一起出發，前往更大的世界。沿途，我們遇到了許多新奇的生物和景色，小王子對每一件事情都充滿著好奇和興趣。

我們來到了一個美麗的花園裡，小王子看到那些美麗的花朵就說：“這些花朵太漂亮了！我想給我的花一個嘴套子，讓它能夠說話。”我笑著說：“你知道，我也曾經畫過一幅猴面包樹的圖片，它看起來很像白菜呢！”小王子又開始笑了。

我們繼續前行，遇到了許多新奇的事情，小王子對每一件事情都充滿著好奇和興趣。最後，我們來到了一個美麗的大海邊，小王子說：“這裡太漂亮了！我想給我的小羊一個嘴套子，讓它能夠說話。”我笑著說：“你知道，我也曾經畫過一幅狐狸的圖片，它的耳朵看起來很像犄角呢！”小王子又開始笑了。

我們繼續前行，直到最後，小王子對這個世界充滿著好奇和興趣。他說：“我想再去看看外面的世界有多大多美麗。”於是，我們決定一起出發，前往更大的世界。


In [21]:
print(store)

{'1': InMemoryHistory(messages=[AIMessage(content='hello', additional_kwargs={}, response_metadata={})]), 'foo': InMemoryHistory(messages=[HumanMessage(content='根據以下內容回答問題：\n圖，可以制作一份 WWW 版的《小王子》主頁，那就更好了。 \n \n    《小王子》 是他作品中比較獨特的一篇，也是最著名的一篇。是一本“為大 \n人們寫的童話故事”。我很喜歡它，從小學看到大學。確實，它給大人們看的， \n理解它必須是“大人”才行。可是，我多么希望我還是一個小王子一樣的孩子… \n \n                                       Loking 錄入于 3.15.1997 \n \n◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎\n◎\n\n圖，可以制作一份 WWW 版的《小王子》主頁，那就更好了。 \n \n    《小王子》 是他作品中比較獨特的一篇，也是最著名的一篇。是一本“為大 \n人們寫的童話故事”。我很喜歡它，從小學看到大學。確實，它給大人們看的， \n理解它必須是“大人”才行。可是，我多么希望我還是一個小王子一樣的孩子… \n \n                                       Loking 錄入于 3.15.1997 \n \n◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎\n◎\n\n圖，可以制作一份 WWW 版的《小王子》主頁，那就更好了。 \n \n    《小王子》 是他作品中比較獨特的一篇，也是最著名的一篇。是一本“為大 \n人們寫的童話故事”。我很喜歡它，從小學看到大學。確實，它給大人們看的， \n理解它必須是“大人”才行。可是，我多么希望我還是一個小王子一樣的孩子… \n \n                                       Loking 錄入于 3.15.1997 \n \n◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎◎\n◎\n\n◎ \n \n                          錄  入  后  記

--
以下不具記憶內容
--

In [12]:
# Setup the Retriever
from langchain.chains import RetrievalQA

# 建立 RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ollama_llm,
    retriever=vector_db.as_retriever(),
)


In [13]:
qa_chain.invoke('你看過經典故事小王子嘛？')

{'query': '你看過經典故事小王子嘛？', 'result': '是的，我看過《小王子》。'}

In [14]:
qa_chain.invoke('啟蒙了你什麼？')

{'query': '啟蒙了你什麼？',
 'result': '啟蒙了你什麼？\n\n根據文本，我們可以看到這個問題的答案是在第二段中提到：地理學幫了我很大的忙。因此，答案應該是：\n\n地理學'}

In [15]:
qa_chain.invoke('你覺得小王子是個怎樣的人？')

{'query': '你覺得小王子是個怎樣的人？',
 'result': '我不太確定能夠給出一個正確的答案，因為這個問題需要對文本進行分析和理解。然而，根據提供的文本，我可以試著給出一些觀察：\n\n小王子似乎是一個敏感、浪漫且有想法的人。他對他遇到的那個人（星星）感到留戀，並認為只有這個人不會使他感到荒唐可笑。這可能表明他是一個比較理想主義和浪漫主義的人。\n\n然而，他也顯示出一些自我懷疑和缺乏勇氣的特徵。他說自己沒有勇氣承認他的真正感受，似乎是因為他害怕被別人嘲笑或不理解。這可能表明他是一個比較內向和脆弱的人。\n\n總的來說，小王子似乎是一個複雜且多面性的角色，他有著理想主義、浪漫主義和自我懷疑等特徵。'}

In [22]:
qa_chain.invoke('用你自己本身的創意，幫我寫出小王子的續集')

{'query': '用你自己本身的創意，幫我寫出小王子的續集',
 'result': 'I don\'t know. The provided text does not contain any information about the story of "The Little Prince" or its characters, so I\'m unable to write a continuation of the story based on my own creativity.'}